Peer-graded Assignment: Segmenting and Clustering Neighborhoods in Toronto

List of neighbourhoods in Toronto- webscraping
<br>
Part 1:

In [1]:
from bs4 import BeautifulSoup
import requests
import pandas as pd


In [2]:
url='https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M'
data  = requests.get(url).text

In [3]:
soup = BeautifulSoup(data,"html5lib")

In [44]:

table_contents=[]
table=soup.find('table')
for row in table.findAll('td'):
    cell = {}
    if row.span.text=='Not assigned':
        pass
    else:
        cell['PostalCode'] = row.p.text[:3]
        cell['Borough'] = (row.span.text).split('(')[0]
        cell['Neighborhood'] = (((((row.span.text).split('(')[1]).strip(')')).replace(' /',',')).replace(')',' ')).strip(' ')
        table_contents.append(cell)

# print(table_contents)
df=pd.DataFrame(table_contents)
df['Borough']=df['Borough'].replace({'Downtown TorontoStn A PO Boxes25 The Esplanade':'Downtown Toronto Stn A',
                                             'East TorontoBusiness reply mail Processing Centre969 Eastern':'East Toronto Business',
                                             'EtobicokeNorthwest':'Etobicoke Northwest','East YorkEast Toronto':'East York/East Toronto',
                                             'MississaugaCanada Post Gateway Processing Centre':'Mississauga'})
df.shape

(103, 3)

In [45]:
df2= df.sort_values(by=['PostalCode'])
df2

,PostalCode,Borough,Neighborhood
6,M1B,Scarborough,"Malvern, Rouge"
12,M1C,Scarborough,"Rouge Hill, Port Union, Highland Creek"
18,M1E,Scarborough,"Guildwood, Morningside, West Hill"
22,M1G,Scarborough,Woburn
26,M1H,Scarborough,Cedarbrae
...,...,...,...
64,M9N,York,Weston
70,M9P,Etobicoke,Westmount
77,M9R,Etobicoke,"Kingsview Village, St. Phillips, Martin Grove ..."
89,M9V,Etobicoke,"South Steeles, Silverstone, Humbergate, Jamest..."


<br>

Part 2:


In [60]:
latitute_csv= "Geospatial_Coordinates.csv"
latitute_df=pd.read_csv(latitute_csv)
latitute_df=latitute_df.rename(columns=({"Postal Code" :'PostalCode'}))
latitute_df
df3= pd.merge(df2, latitute_df, on='PostalCode')
df3


North York                24
Downtown Toronto          17
Scarborough               17
Etobicoke                 11
Central Toronto            9
West Toronto               6
York                       5
East York                  4
East Toronto               4
East York/East Toronto     1
Etobicoke Northwest        1
East Toronto Business      1
Queen's Park               1
Mississauga                1
Downtown Toronto Stn A     1
Name: Borough, dtype: int64

<br>
Part 3:

In [123]:
import folium
import numpy as np
import matplotlib.cm as cm
import matplotlib.colors as colors
from geopy.geocoders import Nominatim
import json
from pandas.io.json import json_normalize
from sklearn.cluster import KMeans


In [124]:
#Toronto Latitute and Longitude

Toronto_map= 'Toronto, ON'
geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(Toronto_map)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(Toronto_map,latitude, longitude))


The geograpical coordinate of Toronto, ON are 43.6534817, -79.3839347.


In [125]:
Toronto_map = folium.Map(location=[latitude, longitude], zoom_start=10)


for lat, lon, borough in zip(df3['Latitude'],df3['Longitude'],df3['Borough']):
    label ='{}'.format(borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius= 5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='skyblue',
        fill_opacity=0.7,
        parse_html=False).add_to(Toronto_map)
    
Toronto_map

In [126]:
df3['Borough'].value_counts()

North York                24
Downtown Toronto          17
Scarborough               17
Etobicoke                 11
Central Toronto            9
West Toronto               6
York                       5
East York                  4
East Toronto               4
East York/East Toronto     1
Etobicoke Northwest        1
East Toronto Business      1
Queen's Park               1
Mississauga                1
Downtown Toronto Stn A     1
Name: Borough, dtype: int64


To simplify the assignment a little. Will be only using location in Downtown Toronto Area from this point

In [127]:
Downtown_Toronto =df3[df3['Borough']=='Downtown Toronto'].reset_index(drop=True)
Downtown_Toronto.drop('Borough', axis=1,inplace=True)
Downtown_Toronto

,PostalCode,Neighborhood,Latitude,Longitude
0,M4W,Rosedale,43.679563,-79.377529
1,M4X,"St. James Town, Cabbagetown",43.667967,-79.367675
2,M4Y,Church and Wellesley,43.665860,-79.383160
3,M5A,"Regent Park, Harbourfront",43.654260,-79.360636
4,M5B,"Garden District, Ryerson",43.657162,-79.378937
5,M5C,St. James Town,43.651494,-79.375418
6,M5E,Berczy Park,43.644771,-79.373306
7,M5G,Central Bay Street,43.657952,-79.387383
8,M5H,"Richmond, Adelaide, King",43.650571,-79.384568
9,M5J,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752


In [128]:
#Downtown Toronto geographical coordinates
Downtown_Toronto_map= 'Downtown Toronto, ON'
dt_geolocator = Nominatim(user_agent="ny_explorer")
dt_location = dt_geolocator.geocode(Downtown_Toronto_map)
dt_latitude = dt_location.latitude
dt_longitude = dt_location.longitude
print('The geograpical coordinate of {} are {}, {}.'.format(Downtown_Toronto_map,dt_latitude, dt_longitude))


The geograpical coordinate of Downtown Toronto, ON are 43.6563221, -79.3809161.


In [129]:
Downtown_Toronto_map= folium.Map(location=[dt_latitude, dt_longitude], zoom_start=12.5)

for dt_lat, dt_lon, label in zip (Downtown_Toronto['Latitude'], Downtown_Toronto['Longitude'], Downtown_Toronto['Neighborhood']):
    label= folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [dt_lat, dt_lon],
    radius=5,
    popup=label,
    color= 'blue',
    fill=True,
    fill_color='skyblue',
    fill_opacity=0.7,
    parese_html=False).add_to(Downtown_Toronto_map)
    

Downtown_Toronto_map

In [130]:
# @hidden_cell
CLIENT_ID = 'XDAQUY2SLNXUSHSTFBD4FHNOWY0NFVJRNKK31P403RCDZI4Y' # your Foursquare ID
CLIENT_SECRET = 'DLPE101NGPOLJEOMCPT3XFWPF1XX1LWAY0Z3ESM2MXEIIOIH' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version
LIMIT = 100 # A default Foursquare API limit value

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: XDAQUY2SLNXUSHSTFBD4FHNOWY0NFVJRNKK31P403RCDZI4Y
CLIENT_SECRET:DLPE101NGPOLJEOMCPT3XFWPF1XX1LWAY0Z3ESM2MXEIIOIH


In [131]:
Downtown_Toronto.loc[0,'Neighborhood']
Downtown_latitude = Downtown_Toronto.loc[0,'Latitude']
Downtown_longitude = Downtown_Toronto.loc[0,'Longitude']
Downtown_name = Downtown_Toronto.loc[0,'Neighborhood']

In [132]:
# @hidden_cell
LIMIT = 100 # limit of number of venues returned by Foursquare API

radius = 500 # define radius

 # create URL
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID, 
    CLIENT_SECRET, 
    VERSION, 
    Downtown_latitude, 
    Downtown_longitude, 
    radius, 
    LIMIT)
url # display URL

'https://api.foursquare.com/v2/venues/explore?&client_id=XDAQUY2SLNXUSHSTFBD4FHNOWY0NFVJRNKK31P403RCDZI4Y&client_secret=DLPE101NGPOLJEOMCPT3XFWPF1XX1LWAY0Z3ESM2MXEIIOIH&v=20180605&ll=43.6795626,-79.37752940000001&radius=500&limit=100'

In [133]:
results = requests.get(url).json()

In [134]:
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

In [135]:
venues = results['response']['groups'][0]['items']
    
nearby_venues = json_normalize(venues) # flatten JSON

# filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']
nearby_venues =nearby_venues.loc[:, filtered_columns]

# filter the category for each row
nearby_venues['venue.categories'] = nearby_venues.apply(get_category_type, axis=1)

# clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

<ipython-input-135-561c05f0fdd1>:3: FutureWarning: pandas.io.json.json_normalize is deprecated, use pandas.json_normalize instead
  nearby_venues = json_normalize(venues) # flatten JSON


,name,categories,lat,lng
0,Rosedale Park,Playground,43.682328,-79.378934
1,Whitney Park,Park,43.682036,-79.373788
2,Alex Murray Parkette,Park,43.678300,-79.382773
3,Milkman's Lane,Trail,43.676352,-79.373842


In [136]:
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

4 venues were returned by Foursquare.


In [137]:
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [138]:
Downtown_venues = getNearbyVenues(names=Downtown_Toronto['Neighborhood'], latitudes=Downtown_Toronto['Latitude'],
                                  longitudes=Downtown_Toronto['Longitude']
                                 )

Rosedale
St. James Town, Cabbagetown
Church and Wellesley
Regent Park, Harbourfront
Garden District, Ryerson
St. James Town
Berczy Park
Central Bay Street
Richmond, Adelaide, King
Harbourfront East, Union Station, Toronto Islands
Toronto Dominion Centre, Design Exchange
Commerce Court, Victoria Hotel
University of Toronto, Harbord
Kensington Market, Chinatown, Grange Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
First Canadian Place, Underground city
Christie


In [139]:
Downtown_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Rosedale,43.679563,-79.377529,Rosedale Park,43.682328,-79.378934,Playground
1,Rosedale,43.679563,-79.377529,Whitney Park,43.682036,-79.373788,Park
2,Rosedale,43.679563,-79.377529,Alex Murray Parkette,43.678300,-79.382773,Park
3,Rosedale,43.679563,-79.377529,Milkman's Lane,43.676352,-79.373842,Trail
4,"St. James Town, Cabbagetown",43.667967,-79.367675,Cabbagetown Brew,43.666923,-79.369289,Café


In [140]:
Downtown_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Berczy Park,57,57,57,57,57,57
"CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport",17,17,17,17,17,17
Central Bay Street,66,66,66,66,66,66
Christie,16,16,16,16,16,16
Church and Wellesley,80,80,80,80,80,80
"Commerce Court, Victoria Hotel",100,100,100,100,100,100
"First Canadian Place, Underground city",100,100,100,100,100,100
"Garden District, Ryerson",100,100,100,100,100,100
"Harbourfront East, Union Station, Toronto Islands",100,100,100,100,100,100


In [141]:
print('There are {} uniques categories.'.format(len(Downtown_venues['Venue Category'].unique())))

There are 202 uniques categories.


In [142]:
# one hot encoding
Downtown_onehot = pd.get_dummies(Downtown_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
Downtown_onehot['Neighborhood'] = Downtown_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [Downtown_onehot.columns[-1]] + list(Downtown_onehot.columns[:-1])
Downtown_onehot = Downtown_onehot[fixed_columns]

Downtown_onehot.head()

,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


Getting the mean of the frequency of occurrence for each category

In [143]:
Downtown_grouped = Downtown_onehot.groupby('Neighborhood').mean().reset_index()
Downtown_grouped



,Neighborhood,Yoga Studio,Adult Boutique,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,...,Tea Room,Thai Restaurant,Theater,Theme Restaurant,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar
0,Berczy Park,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.017544,0.000000,0.0000,0.00,0.00,0.017544,0.00000,0.000000,0.000000
1,"CN Tower, King and Spadina, Railway Lands, Har...",0.000000,0.0000,0.058824,0.058824,0.058824,0.117647,0.117647,0.117647,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.00,0.00,0.000000,0.00000,0.000000,0.000000
2,Central Bay Street,0.015152,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.015152,0.015152,0.000000,0.0000,0.00,0.00,0.015152,0.00000,0.000000,0.015152
3,Christie,0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.0000,0.00,0.00,0.000000,0.00000,0.000000,0.000000
4,Church and Wellesley,0.025000,0.0125,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.012500,...,0.000000,0.012500,0.012500,0.0125,0.00,0.00,0.000000,0.00000,0.000000,0.000000
5,"Commerce Court, Victoria Hotel",0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,...,0.010000,0.010000,0.000000,0.0000,0.00,0.00,0.020000,0.00000,0.000000,0.010000
6,"First Canadian Place, Underground city",0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.020000,...,0.010000,0.000000,0.010000,0.0000,0.00,0.01,0.010000,0.00000,0.000000,0.010000
7,"Garden District, Ryerson",0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.010000,0.010000,0.020000,0.0000,0.00,0.00,0.000000,0.01000,0.010000,0.010000
8,"Harbourfront East, Union Station, Toronto Islands",0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.010000,0.010000,0.000000,0.0000,0.00,0.01,0.000000,0.00000,0.000000,0.010000
9,"Kensington Market, Chinatown, Grange Park",0.000000,0.0000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.015152,0.000000,0.000000,0.0000,0.00,0.00,0.045455,0.00000,0.045455,0.015152


Getting the top 5 most common venues

In [144]:
num_top_venues = 5

for hood in Downtown_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = Downtown_grouped[Downtown_grouped['Neighborhood'] == hood].T.reset_index()
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Berczy Park----
            venue  freq
0     Coffee Shop  0.09
1    Cocktail Bar  0.05
2          Bakery  0.05
3        Beer Bar  0.04
4  Farmers Market  0.04


----CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport----
              venue  freq
0    Airport Lounge  0.12
1   Airport Service  0.12
2  Airport Terminal  0.12
3   Harbor / Marina  0.06
4             Plane  0.06


----Central Bay Street----
                 venue  freq
0          Coffee Shop  0.17
1       Sandwich Place  0.06
2   Italian Restaurant  0.05
3                 Café  0.05
4  Japanese Restaurant  0.03


----Christie----
           venue  freq
0  Grocery Store  0.25
1           Café  0.19
2           Park  0.12
3     Baby Store  0.06
4    Candy Store  0.06


----Church and Wellesley----
                 venue  freq
0          Coffee Shop  0.08
1  Japanese Restaurant  0.06
2     Sushi Restaurant  0.06
3           Restaurant  0.04
4              Gay Bar  0.0

Generating new dataframe with the top 5 venues for each neighborhood are showed

In [146]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

num_top_venues = 5

indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = Downtown_grouped['Neighborhood']

for ind in np.arange(Downtown_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Downtown_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Berczy Park,Coffee Shop,Bakery,Cocktail Bar,Beer Bar,Cheese Shop
1,"CN Tower, King and Spadina, Railway Lands, Har...",Airport Lounge,Airport Service,Airport Terminal,Boat or Ferry,Coffee Shop
2,Central Bay Street,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Restaurant
3,Christie,Grocery Store,Café,Park,Nightclub,Candy Store
4,Church and Wellesley,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant


Cluster Neighborhoods

In [160]:
# set number of clusters
kclusters = 5

Downtown_grouped_clustering = Downtown_grouped.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(Downtown_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([1, 0, 1, 2, 1, 1, 1, 1, 1, 3])

Making a new datadrame showing Geographical location, cluster lables, and top 5 venues of each neighborhood at the sametime

In [ ]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
Downtown_merged= Downtown_Toronto
Downtown_merged= Downtown_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')


In [164]:
Downtown_merged.head(10)

,PostalCode,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,M4W,Rosedale,43.679563,-79.377529,4,Park,Trail,Playground,Wine Bar,Cupcake Shop
1,M4X,"St. James Town, Cabbagetown",43.667967,-79.367675,1,Coffee Shop,Café,Pub,Bakery,Pizza Place
2,M4Y,Church and Wellesley,43.665860,-79.383160,1,Coffee Shop,Sushi Restaurant,Japanese Restaurant,Gay Bar,Restaurant
3,M5A,"Regent Park, Harbourfront",43.654260,-79.360636,1,Coffee Shop,Bakery,Café,Park,Pub
4,M5B,"Garden District, Ryerson",43.657162,-79.378937,1,Coffee Shop,Clothing Store,Sandwich Place,Café,Hotel
5,M5C,St. James Town,43.651494,-79.375418,1,Café,Coffee Shop,Cocktail Bar,Gastropub,Restaurant
6,M5E,Berczy Park,43.644771,-79.373306,1,Coffee Shop,Bakery,Cocktail Bar,Beer Bar,Cheese Shop
7,M5G,Central Bay Street,43.657952,-79.387383,1,Coffee Shop,Sandwich Place,Café,Italian Restaurant,Restaurant
8,M5H,"Richmond, Adelaide, King",43.650571,-79.384568,1,Coffee Shop,Café,Thai Restaurant,Restaurant,Gym
9,M5J,"Harbourfront East, Union Station, Toronto Islands",43.640816,-79.381752,1,Coffee Shop,Café,Hotel,Restaurant,Scenic Lookout


Visualize the clusters

In [169]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=13)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Downtown_merged['Latitude'], Downtown_merged['Longitude'], Downtown_merged['Neighborhood'], Downtown_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ', Cluster: ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

Result:

Food businesses (coffee shop, bakery, restuarants, and etc) play a dominant role in Downtown Toronto Area.
Most businesses are located at the South East part of Downtwon Toronto